#start


In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:

import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:


# Paths
output_dir = '/content/drive/MyDrive/Tomato_dataset/cnn_crops'

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    zoom_range=0.2
)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load datasets
datasets_dict = {
    'ripe': {
        'train': train_datagen.flow_from_directory(
            os.path.join(output_dir, 'ripe/train'),
            target_size=(224,224),
            batch_size=16,
            class_mode='categorical'
        ),
        'val': val_datagen.flow_from_directory(
            os.path.join(output_dir, 'ripe/val'),
            target_size=(224,224),
            batch_size=16,
            class_mode='categorical'
        )
    },
    'green': {
        'train': train_datagen.flow_from_directory(
            os.path.join(output_dir, 'green/train'),
            target_size=(224,224),
            batch_size=16,
            class_mode='categorical'
        ),
        'val': val_datagen.flow_from_directory(
            os.path.join(output_dir, 'green/val'),
            target_size=(224,224),
            batch_size=16,
            class_mode='categorical'
        )
    }
}

# Define CNN model
def create_cnn(num_classes):
    model = Sequential([
        Conv2D(16, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
        MaxPooling2D(2,2),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
# Cell 2: Train Ripe Tomato CNN

ripe_train = datasets_dict['ripe']['train']
ripe_val = datasets_dict['ripe']['val']
num_classes = len(ripe_train.class_indices)

ripe_model = create_cnn(num_classes)

ripe_save_path = '/content/drive/MyDrive/Tomato_dataset/models/ripe_cnn.h5'

history_ripe = ripe_model.fit(
    ripe_train,
    validation_data=ripe_val,
    epochs=20,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

ripe_model.save(ripe_save_path)
print(f"Ripe CNN trained and saved to {ripe_save_path}")


In [ ]:
# Cell 3: Train Green Tomato CNN

green_train = datasets_dict['green']['train']
green_val = datasets_dict['green']['val']
num_classes = len(green_train.class_indices)

green_model = create_cnn(num_classes)

green_save_path = '/content/drive/MyDrive/Tomato_dataset/models/green_cnn.h5'

history_green = green_model.fit(
    green_train,
    validation_data=green_val,
    epochs=20,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

green_model.save(green_save_path)
print(f"Green CNN trained and saved to {green_save_path}")
